### comparing xgboost model with 7 features vs all 985 features

In [1]:
import sys
sys.path.append('..')

In [2]:
from core.dataset import *
import seaborn as sns
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

#### identifying all feature columns corresponding to 7 selected features

##### Previously: ~ 62% accuracy when trained with all features

In [3]:
load_dir = 'D:/Data/hoffmanlab/featureselection/data/'
ligands = ['CpG', 'FLA', 'FSL', 'LPS', 'P3K', 'PIC', 'R84', 'TNF']
sheet_type = 'am'

In [4]:
dataset = Data(load_dir, ligands, sheet_type, merge=True, numpy=False)

##### Extract the 7 seven features from this dataset

In [5]:
dataset.columns

Index([  'time_series_1',   'time_series_2',   'time_series_3',
         'time_series_4',   'time_series_5',   'time_series_6',
         'time_series_7',   'time_series_8',   'time_series_9',
        'time_series_10',
       ...
        'fold_change_92',  'fold_change_93',  'fold_change_94',
        'fold_change_95',  'fold_change_96',  'fold_change_97',
        'fold_change_98', 'max_fold_change',       'max_value',
                       0],
      dtype='object', length=985)

In [6]:
extracted_features = [dataset['integrals_pos_38'], dataset['min_trough2peak'], dataset['integrals_pos_11'], dataset['integrals_40'],
                      dataset['integrals_pos_14'], dataset['time_series_2'], dataset['integrals_pos_12']]

In [7]:
data = pd.DataFrame()

In [15]:
data = pd.concat(extracted_features, axis=1).to_numpy()
labels = dataset.iloc[:, [984]].to_numpy()

In [16]:
labels.shape

(14376, 1)

In [18]:
X = np.hstack([data, labels])

In [19]:
labels = np.reshape(labels, (-1, ))

In [20]:
X.shape

(14376, 8)

In [21]:
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.1, random_state=42)

#### training the model on the top 7 features

In [22]:
model = xgb.XGBClassifier()

In [23]:
model.fit(X_train, y_train)

C:\Users\minha\miniconda3\envs\peep\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:19:24] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [24]:
pred = model.predict(X_val)

In [25]:
pred

array([6, 6, 7, ..., 2, 0, 7])

In [26]:
acc = 0
for i, j in zip(pred, y_val):
    if i == j:
        acc += 1
print(f'Accuracy: {acc / len(y_val)}')

Accuracy: 0.480528511821975


#### Accuracy went from 62% -> 48% when trained on 7 features vs 985

In [27]:
### ignore everything below this line for now

In [28]:
# print(dataset.columns.get_loc('power_1'), dataset.columns.get_loc('power_125'))
# power = dataset.iloc[:, [i for i in range(650, 775)]]
# print(dataset.columns.get_loc('max_value'))
# max_value = dataset.iloc[:, [983]]
# print(dataset.columns.get_loc('ipt_1'), dataset.columns.get_loc('ipt_15'))
# ipt = dataset.iloc[:, [i for i in range(449, 464)]]
# print(dataset.columns.get_loc('num_peaks'))
# num_peaks = dataset.iloc[:, [464]]
# print(dataset.columns.get_loc('valley_amps_1'), )

#### Training a model on all time series per feature for top 7 features
* integrals_pos
* min_trough2peak
* integrals
* envelope
* oscpower
* fold_change
* valley_amps

## To do
* hyperparameter tuning
* try a simpler model, look at coefficients
* look at correlation between features
* take a subset of features...
* use that correlation to identify redundant features, clean out the features used in the 'reduced-training' model
* downsample random features, see what happens
* visualization

## Figure out a way to remove redundant features or combine them
* if the top feature is like integral 38, 37, 36
* take maximum, average, ... etc?

## Once we have xyz amount of features
* visualizations: distance between all features vs standard matrix for selected features
* dimensionality reduction
* find foundation of xyz features, give or take a little and see what happens

## Graphs
* could partition the giant graph of feature importance into multiple little ones so you can actually see what's going on lol